In [1]:
import pandas as pd
import scipy.sparse

from recommender.testing.custom_metric_utils import calculate_metric_scores
from recommender.testing.custom_metric_utils import create_recommendations_profiles_embeddings
from recommender.tools.lightfm_utils import LightFMTests

In [2]:
DATA_ROOT = '../../data'
THREADS = 8

## Data loading

In [3]:
full_df = pd.read_csv(f'{DATA_ROOT}/ratings_all_implicit.csv.gz')
train_df = pd.read_csv(f'{DATA_ROOT}/ratings_train_implicit.csv.gz')
games_df = pd.read_json(f'{DATA_ROOT}/bgg_GameItem.jl', lines=True)[[
    'name', 'bgg_id', 'mechanic', 'category', 'complexity',
    'max_players_best', 'min_players_best', 'max_players_rec', 'min_players_rec'
]]

features_names = pd.read_csv(f'{DATA_ROOT}/game_features_names.csv.gz').values.flatten()
game_features = pd.read_csv(f'{DATA_ROOT}/game_features.csv.gz')

model = pd.read_pickle('../notebooks_data/custom_model.pkl')
dataset = pd.read_pickle(f'{DATA_ROOT}/dataset.pkl')
item_features = pd.read_pickle(f'{DATA_ROOT}/item_features.pkl')

train_interactions = scipy.sparse.load_npz(f'{DATA_ROOT}/train_interactions.npz')
test_interactions = scipy.sparse.load_npz(f'{DATA_ROOT}/test_interactions.npz')

In [4]:
test_users_profiles = pd.read_pickle(f'{DATA_ROOT}/test_users_profiles.pkl')

## Data processing

In [5]:
mechanics_names = features_names[:20]
categories_names = features_names[20:40]

In [6]:
test_users_profiles.index = test_users_profiles.index.map(dataset.mapping()[0])
test_users_profiles = test_users_profiles.sort_index()

In [7]:
test_users_profiles.values[:, :40] *= 3

## Evaluation

In [8]:
users_ids = test_users_profiles.index.values
users_ids.sort()

In [9]:
test_class = LightFMTests(model, train_df, dataset, train_interactions, test_interactions, item_features)
num_recs = 5
print("Creating recommendation profiles")
recommendation_profiles = create_recommendations_profiles_embeddings(
    test_class.get_top_n(num_recs, users_ids), num_recs, games_df, categories_names, mechanics_names, 10, show_progress=True
)
recommendation_profiles.index = recommendation_profiles.index.map(dataset.mapping()[0])
recommendation_profiles = recommendation_profiles.sort_index()
metric = calculate_metric_scores(recommendation_profiles, test_users_profiles.loc[users_ids])

Creating recommendation profiles


100%|██████████| 76543/76543 [04:59<00:00, 255.35it/s]


In [10]:
metric

array([1.51037832, 1.46374173, 0.70099428, 0.76516337, 4.44027771])